In [7]:
from dotenv import load_dotenv
import os
from pathlib import Path
import psycopg2

import pandas as pd

In [2]:
# Getting the database variables from the .env file
load_dotenv()

DB_KEY = os.getenv("DB_KEY")
DB_NAME = os.getenv("DB_NAME")

# Connecting to the database
conn = psycopg2.connect(
    dbname='PEDE_PASSOS',
    user=DB_NAME,
    host='localhost',
    port='5432',
    password=DB_KEY)

In [10]:
# Reading the data from the database
data = pd.read_sql("SELECT * FROM pede_passos.students_grades", con=conn)

db_data_path = Path('../data/interim/students_grades.csv')

data.to_csv(db_data_path, index=False)

/var/folders/s8/p68_rlhj54bb1q_xvzdsc1m40000gn/T/ipykernel_74089/281504888.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql("SELECT * FROM pede_passos.students_grades", con=conn)
